<a href="https://colab.research.google.com/github/simgyojin/2020-Bigdata-Finance-Challenge/blob/main/%EC%99%B8%EB%B6%80%EB%8D%B0%EC%9D%B4%ED%84%B0/%EC%BD%94%EB%93%9C/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC_%EB%A9%B4%EC%A0%81_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#위키백과에서 면적 크롤링
###행정동 별 면적을 얻기 위해 위키백과에 나와있는 면적을 토대로 행정동 별 인구 밀도 계산

허점->001동, 00제3동 과 같은 행정동 이름 고려X

In [ ]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from google.colab import files
import re
get_num = re.compile('\d+')
get_eng = re.compile('[a-z]+')
#files.upload()

In [282]:
#전국 행정동 이름이 나와있는 csv파일
district_df = pd.read_csv('행정동_20200814.csv', encoding='utf-8')
district_list = list(district_df['읍면동명'])

#인구밀도 가공 csv파일
population_df = pd.read_csv('인구밀도_가공_최종.csv', encoding='utf-8')

#필요없는 columns제거
population_df.drop(['면적(km^2)','인구밀도'], axis='columns', inplace=True)

In [229]:
#중복되는 행정동에는 시군구명 붙이기
real_district_list=[]
for n, district in enumerate(district_list):
  if str(district) != 'nan':    #nan인 경우 제외
    if district_list.count(district) >=2:   #행정동이 동명인 경우
      if len(str(district_df['시군구명'][n]).split()) >= 2:  #'수원시 팔달구' 같이 시군구명 띄어쓰기가 되어 있으면 수원시만 가져옴
        real_district_list.append('{} ({}),{}'.format(district,district_df['시군구명'][n].split()[0],n))
      else:
        if str(district_df['시도명'][n])[-1] == '시':       #특별시, 자치시와 같은 경우 위키백과에서 제대로된 데이터 가져오기 위해 사용
          real_district_list.append('{} ({}),{}'.format(district,str(district_df['시도명'][n]).split()[0][:2],n))
        else:
          real_district_list.append('{} ({}),{}'.format(district,district_df['시군구명'][n],n))
    else:
      real_district_list.append('{},{}'.format(district,n))

In [230]:
#위키 백과에서 행정동 면적을 크롤링 하는 코드
#인구밀도가 나와있는 지역도 있지만, 없는 지역도 존재하여 면적만 크롤링
def get_area(district_name):
  url = 'https://ko.wikipedia.org/wiki/{}'.format(district_name)
  driver.get(url)
  area = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table/tbody/tr[2]/td').text
  area2 = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table/tbody/tr[3]/td').text
  area3 = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table/tbody/tr[4]/td').text
  area4 = driver.find_element_by_xpath('/html/body/div[3]/div[3]/div[5]/div[1]/table/tbody/tr[5]/td').text
  area_list = [area4,area3,area2,area]
  real_area=0
  for aa in area_list:
    if aa.count('km') >=1:
      real_area=aa
  return real_area.split()[0]

In [ ]:
#chome driver사용
'''
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
'''
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

#지역과 지역의 면적을 담을 dictionary
district_area_dic={}

#district_list내의 지역의 면적을 크롤링 real_district_list의 length:3558
for n, district in enumerate(real_district_list):
  ddistrint = district.split(',')[0]
  nn = int(district.split(',')[1])
  dd_list = []
  try:
    aa = get_area(ddistrint)
    dd_list.append(aa)
    dd_list.append(district_df['행정동코드'][nn])
    print('{}번째: {} is doing-{}'.format(n, ddistrint,aa))
  except:
    try:
      area = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/ul/li[1]/b/a')
      last_url = area.get_attribute("href").split('/')[-1]
      aa = get_area(ddistrint)
      dd_list.append(aa)
      dd_list.append(district_df['행정동코드'][nn])
      print('{}번째: {} is doing-{}'.format(n, ddistrint,aa))
    except:
      try:
        area = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div[1]/ul/li[1]/a[3]')
        last_url = area.get_attribute("href").split('/')[-1]
        aa = get_area(ddistrint)
        dd_list.append(aa)
        dd_list.append(district_df['행정동코드'][nn])
        print('{}번째: {} is doing-{}'.format(n, ddistrint,aa))
      except:
        print('{}번째: {} is None'.format(n, ddistrint))
  district_area_dic[ddistrint]=dd_list

In [236]:
#district_area_dic의 빈 행정동은 제거
district_area_dd={}
for i in district_area_dic:
  if len(district_area_dic[i]) != 0:
    district_area_dd[i.split(',')[0]]=district_area_dic[i]

In [251]:
#dataframe 형식으로 변환하여 column명 생성
district_dataframe = pd.DataFrame(district_area_dd).T
district_dataframe.columns=['면적','행정동코드']

In [276]:
#전처리 잘못된 두가지 바꿈
for i in range(len(district_dataframe)):
  dd = get_eng.findall(district_dataframe['면적'][i])
  if len(dd)!=0:
    want = get_num.findall(district_dataframe['면적'][i])
    if len(want) >=1:
      ww = '.'.join(want[:2])
      district_dataframe['면적'][i] = float(ww)
  else:
    try:
      district_dataframe['면적'][i] = float(district_dataframe['면적'][i])
    except:
      district_dataframe['면적'][i] = 63.82

In [283]:
#df merge
merge_df = pd.merge(population_df,district_dataframe, on='행정동코드')
merge_df['2019인구밀도'] = merge_df['2019인구수']/merge_df['면적']
merge_df['2020인구밀도'] = merge_df['2020인구수']/merge_df['면적']

In [285]:
#csv파일로 저장
merge_df.to_csv('2019,2020 인구밀도 계산완료.csv',index=False, encoding='utf-8')

#file다운로드
files.download('2019,2020 인구밀도 계산완료.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>